In [1]:
import numpy as np

from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, RNN, LSTM
from torch.nn.functional import softmax, relu

from sklearn.manifold import TSNE
import h5py

In [2]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

False


In [3]:
# Load data
with h5py.File('C:/Users/jovog/OneDrive/Documents/50000_dataset.hdf5', 'r') as hf:
    data = hf['data'][:]
    label = hf['label'][:]
data[:,1] += 1
label[:] += 1

In [4]:
seq = [] #Initalize
lab = []
current_user = -1
for i in range(data.shape[0]): # going through the data and sorting into user sequences
    if data[i,0]!=current_user:
        current_user = data[i,0]
        seq.append([data[i,1]])
        lab.append([label[i]])        
    else:
        seq[-1].append(data[i,1]) # new user line
        lab[-1].append(label[i])
        
seq.sort(key=lambda x:len(x))
lab.sort(key=lambda x:len(x))
seq_length = [len(x) for x in seq]

In [5]:
# Define network
output_size = 17771
num_input = 17771
hidden_size_rnn1 = 1000
hidden_size_lin1 = 1000

hidden_size_rnn2 = 2000
hidden_size_lin2 = 5000

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.one_hot = nn.Embedding(num_input, num_input)
        self.one_hot.weight.data = torch.eye(num_input)
        self.one_hot.weight.detach_()
        
        self.rnn1 = LSTM(input_size=num_input,
                        hidden_size=hidden_size_rnn1,
                        num_layers=1,
                        bidirectional=False,
                        batch_first = True)
        
        #self.lin1 = Linear(in_features=hidden_size_rnn1,
        #                   out_features=hidden_size_lin1,
        #                   bias=False)
        
        #self.rnn2 = LSTM(input_size=hidden_size_lin1,
        #                 hidden_size=hidden_size_rnn2,
        #                 num_layers=1,
        #                 bidirectional=False,
        #                 batch_first=True)
        
        #self.lin2 = Linear(in_features=hidden_size_rnn2,
        #                   out_features=hidden_size_lin2,
        #                   bias=False)
        
        self.l_out = Linear(in_features=hidden_size_rnn1,
                            out_features=output_size,
                            bias=False)
        self.criteria = nn.CrossEntropyLoss()
    
    def forward(self, x):
        out = {}
        x = self.one_hot(x)
        
        x, hn = self.rnn1(x)
        
        #x = relu(self.lin1(x))
        
        
        #x, hn = self.rnn2(x)
        
        #x = relu(self.lin2(x))
        
        out = (self.l_out(x))
        return out
    
    def MyLoss(self, Y_hat, Y):
        # create a mask by filtering out all tokens that ARE NOT the padding token
        mask = (Y > 0).float()
        #if mask[0,-1]==0:
        #    m = len(mask[0,:])-1
        mask = mask.view(-1,1)
        
        Y = Y.view(-1)
        
        Y_hat = Y_hat.view(-1, output_size)
        
        # count how many tokens we have
        #nb_tokens = int(torch.sum(mask).data[0])
        #print('y_hat:',Y_hat.size(),Y_hat)
        #print('y:',Y.size(),Y)
        #print('mask:',mask.size(),mask)
        # pick the values for the label and zero out the rest with the mask
        Y_hat = Y_hat * mask
        #print('2:',Y_hat.size(),Y_hat)
        #print('3:',Y_hat[m,:])
        #print(Y_hat.type())
        #print(Y.type())
        loss = self.criteria(Y_hat,Y)
        return loss
net = Net()
if use_cuda:
    net.cuda()
print(net)

Net(
  (one_hot): Embedding(17771, 17771)
  (rnn1): LSTM(17771, 1000, batch_first=True)
  (l_out): Linear(in_features=1000, out_features=17771, bias=False)
  (criteria): CrossEntropyLoss()
)


In [6]:
{p[0]: p[1].requires_grad for p in net.named_parameters()}

{'one_hot.weight': False,
 'rnn1.weight_ih_l0': True,
 'rnn1.weight_hh_l0': True,
 'rnn1.bias_ih_l0': True,
 'rnn1.bias_hh_l0': True,
 'l_out.weight': True}

In [7]:
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [ ]:
# Train and validation loop
batch_size = 10
net.train()
for i in np.random.choice(len(seq)//batch_size,len(seq)//batch_size, replace=False):
    batch = seq[i*batch_size:(i+1)*batch_size]
    batch_length = seq_length[i*batch_size:(i+1)*batch_size]
    batch_label = lab[i*batch_size:(i+1)*batch_size]
    
    # Zero padding of the batch
    max_len_batch = max(batch_length)
    batch_padded = np.zeros((batch_size,max_len_batch))
    padded_label = np.zeros((batch_size,max_len_batch))
    for j in range(batch_size):
        batch_padded[j,0:batch_length[j]] = batch[j]
        padded_label[j,0:batch_length[j]] = batch_label[j]
    batch_padded = torch.from_numpy(batch_padded).long()
    padded_label = torch.from_numpy(padded_label).long()
    output = net(batch_padded)
    #print('batch_length:',batch_length)
    batch_loss = net.MyLoss(output,padded_label)
    print(batch_loss)
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

tensor(9.7847, grad_fn=<NllLossBackward>)
tensor(9.6105, grad_fn=<NllLossBackward>)
tensor(10.2266, grad_fn=<NllLossBackward>)
tensor(12.5611, grad_fn=<NllLossBackward>)
tensor(11.0495, grad_fn=<NllLossBackward>)
tensor(9.8305, grad_fn=<NllLossBackward>)
tensor(8.4837, grad_fn=<NllLossBackward>)
tensor(8.4375, grad_fn=<NllLossBackward>)
tensor(10.1540, grad_fn=<NllLossBackward>)
tensor(9.9748, grad_fn=<NllLossBackward>)
tensor(13.6747, grad_fn=<NllLossBackward>)
tensor(10.5146, grad_fn=<NllLossBackward>)
tensor(11.0789, grad_fn=<NllLossBackward>)
tensor(11.3230, grad_fn=<NllLossBackward>)
tensor(10.7022, grad_fn=<NllLossBackward>)
tensor(12.1851, grad_fn=<NllLossBackward>)


In [ ]:
print(padded_label.size())